# chap 01 상자로서의 변수

In [154]:
import numpy as np

In [155]:
def as_array(x):
    if np.isscalar(x):
        return np.array(x)
    return x

In [156]:
class Variable:
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError('{}은(는) 지원하지 않습니다.'.format(type(data)))
            
        self.data = data
        self.grad = None
        self.creator = None
        
    def set_creator(self, func):
        self.creator = func
        
    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)
        funcs = [self.creator]
        while funcs:
            f = funcs.pop()
            x, y = f.input, f.output
            x.grad = f.backward(y.grad)
            
            if x.creator is not None:
                funcs.append(x.creator)

# chap 02 변수를 낳는 함수

In [157]:
class Function:
    def __call__(self, input):
        x = input.data
        y = self.forward(x)
        output = Variable(as_array(y))
        output.set_creator(self)
        self.input = input
        self.output = output
        return output
    
    def forward(self, x):
        raise NotImplementedError()
    
    def backward(self, gy):
        raise NotImplementedError()
    
class Square(Function):
    def forward(self, x):
        y = x**2
        return y
    
    def backward(self, gy):
        x = self.input.data
        gx = 2*x*gy
        return gx

# chap 03 함수 연결

In [158]:
class Exp(Function):
    def forward(self, x):
        y = np.exp(x)
        return y
    
    def backward(self, gy):
        x = self.input.data
        gx = np.exp(x)*gy
        return gx

# chap 04 수치 미분

In [159]:
def numerical_diff(f, x, eps=1e-4):
    x0 = Variable(x.data - eps)
    x1 = Variable(x.data + eps)
    y0 = f(x0)
    y1 = f(x1)
    return (y1.data - y0.data)/(2*eps)

# chap 07 역전파 자동화

In [160]:
"""
#Test generator

A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))

a = A(x)
b = B(a)
y = C(b)

# backpropagation
y.grad = np.array(1.0)
y.backward()
print(x.grad)
"""

'\n#Test generator\n\nA = Square()\nB = Exp()\nC = Square()\n\nx = Variable(np.array(0.5))\n\na = A(x)\nb = B(a)\ny = C(b)\n\n# backpropagation\ny.grad = np.array(1.0)\ny.backward()\nprint(x.grad)\n'

# chap 09 함수를 더 편리하게

In [161]:
def square(x):
    return Square()(x)

def exp(x):
    return Exp()(x)
"""
x = Variable(np.array(0.5))
y = square(exp(square(x)))
y.backward()
print(x.grad)"""

'\nx = Variable(np.array(0.5))\ny = square(exp(square(x)))\ny.backward()\nprint(x.grad)'

# chap 10 테스트

In [162]:
import unittest

class SquareTest(unittest.TestCase):
    def test_forward(self):
        x = Variable(np.array(2.0))
        y = square(x)
        expected = np.array(4.0)
        self.assertEqual(y.data, expected)